In [ ]:
import numpy as np
seed = 7
np.random.seed(seed)
import dask.dataframe as dd
import pandas as pd
import pandas.core.algorithms as algos


from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
#from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import os
from scipy.sparse import csr_matrix, hstack

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.layers.advanced_activations import PReLU

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## loading data

In [ ]:
#importing the data
feat_gat_tr = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/gender_age_train.csv')
feat_gat_test = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/gender_age_test.csv')
feat_ph = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/phone_brand_device_model.csv')
# removing duplicate values in phone dataframe so doesn't create problems while joining dataframes
feat_ph=feat_ph.drop_duplicates('device_id',keep='first')

events = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/events.csv',
                     parse_dates=['timestamp'],
                     infer_datetime_format=True,
                     )

feat_ap_eve = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/app_events.csv',
                    dtype={'is_installed':bool, 'is_active':bool})

feat_app_lab = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/app_labels.csv') 
feat_fold_5=pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/folds_5.csv')
feat_fold_10=pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/folds_10.csv') 
feat_lab_cat = pd.read_csv('/content/gdrive/MyDrive/talkingdata-mobile-user-demographics/label_categories.csv')

In [ ]:
import pickle
feat_br_encoder = pickle.load(open('feat_br_encoder.sav','rb')) 

array(['E人E本', 'E派', 'HTC', 'LG', 'LOGO', 'Lovme', 'MIL', 'OPPO', 'PPTV',
       'SUGAR', 'TCL', 'ZOYE', 'ZUK', 'vivo', '一加', '三星', '世纪天元', '世纪星',
       '中兴', '中国移动', '丰米', '乐视', '乡米', '亚马逊', '亿通', '优米', '优语', '优购',
       '先锋', '凯利通', '努比亚', '华为', '华硕', '原点', '台电', '唯比', '唯米', '嘉源', '基伍',
       '夏新', '大Q', '大可乐', '大显', '天宏时代', '天语', '奇酷', '奥克斯', '宏碁', '宝捷讯',
       '富可视', '小杨树', '小米', '尼比鲁', '帷幄', '广信', '康佳', '德卡诺', '德赛', '恒宇丰',
       '惠普', '戴尔', '摩乐', '摩托罗拉', '斐讯', '昂达', '易派', '普耐尔', '智镁', '本为',
       '朵唯', '极米', '果米', '梦米', '欧乐迪', '欧乐酷', '欧博信', '欧奇', '欧新', '欧比',
       '欧沃', '沃普丰', '波导', '海信', '海尔', '爱派尔', '瑞米', '瑞高', '白米', '百加',
       '百立丰', '碟米', '神舟', '米奇', '米歌', '糖葫芦', '糯米', '索尼', '纽曼', '维图', '美图',
       '聆韵', '联想', '至尊宝', '艾优尼', '蓝魔', '虾米', '西米', '西门子', '语信', '诺亚信',
       '诺基亚', '谷歌', '贝尔丰', '赛博宇华', '邦华', '酷比', '酷比魔方', '酷派', '酷珀', '金星数码',
       '金立', '锤子', '长虹', '青橙', '青葱', '飞利浦', '飞秒', '首云', '魅族', '鲜米', '黑米'],
      dtype=object)

In [ ]:

feat_unq_col=np.load('unique_columns.npy')

In [ ]:
def feat_transform(brand,model): 
    #loading data
    
    #making brand feature
    feat_br_encoder = pickle.load(open('feat_br_encoder.sav','rb'))
    feat_l=[] 
    feat_l.append(brand)
    
    
    feat_rp=feat_br_encoder.transform(l)  
    
    feat_n=np.zeros(len(feat_br_encoder.classes_)) 
    
    feat_n[feat_rp[0]]=1 
    feat_sPhone = sparse.csr_matrix(feat_n)  
    print("shape of phone {}".format(feat_sPhone.shape)) 
    
    # making model feature
    feat_mod_encodr = pickle.load(open('feat_mod_encodr.sav','rb'))
    #converting model to labels
    feat_modl=brand+model  
    feat_l=[] 
    feat_l.append(feat_modl)  
    feat_rm=feat_mod_encodr.transform(feat_l)  
    feat_n=np.zeros(len(feat_mod_encodr.classes_)) 
    
    feat_n[feat_rm[0]]=1  
    feat_sMod = sparse.csr_matrix(feat_n) 
    print("shape of model {}".format(feat_sMod.shape))
    #model frequency feature
    mod_freq = pickle.load(open('mod_freq.pkl','rb'))  
    feat_mfreq=mod_freq[model]
    
    feat_mf_encod =pickle.load(open('feat_mf_encod.sav','rb'))
    feat_l=[] 
    feat_l.append(feat_mfreq) 
    feat_rmf=feat_mf_encod.transform(feat_l) 
    feat_nm_freqs = len(feat_mf_encod.classes_) 
    feat_n=np.zeros(feat_nm_freqs) 
    feat_n[feat_rmf[0]]=1 
    feat_sMod_freq = sparse.csr_matrix(feat_n) 
    print("shape of mfreq {}".format(feat_sMod_freq.shape))
    
    #making brand frequency feature 
    feat_br_freq = pickle.load(open('feat_br_freq.pkl','rb'))
    feat_bfreq=feat_br_freq[brand]
    feat_l=[] 
    feat_l.append(feat_bfreq)
    feat_bf_encod = pickle.load(open('feat_bf_encod.sav','rb'))

    feat_rbf=feat_bf_encod.transform(feat_l) 
    feat_nbfreqs = len(feat_bf_encod.classes_) 
    feat_n=np.zeros(feat_nbfreqs) 
    feat_n[feat_rbf[0]]=1 
    feat_sBr_freq = sparse.csr_matrix(feat_n)
    print("shape of brandfreq {}".format(feat_sBr_freq.shape)) 
    
    #making tfidf features for brand and frequency
    
    feat_tfv = pickle.load(open('tfv_noevent.sav','rb'))

    feat_tf=brand+" "+model
    feat_tfidf_test=feat_tfv.transform([feat_tf]) 
    print("shape of tfidf {}".format(feat_tfidf_test.shape))
    return hstack((feat_sPhone,feat_sMod,feat_sMod_freq,feat_sBr_freq,feat_tfidf_test))
    
    
    
    
    

In [ ]:
from scipy import sparse
def feat_trans_eve(brand,model,appid,labelid,timestamps,unique_columns): 
    #loading data to make features
    
    #making brand featurea
    feat_br_encoder = pickle.load(open('feat_br_encoder.sav','rb'))
    feat_l=[] 
    feat_l.append(brand)
    feat_rp=feat_br_encoder.transform(feat_l)  
    
    feat_n=np.zeros(len(feat_br_encoder.classes_)) 
    
    feat_n[feat_rp[0]]=1 
    feat_sPhone = sparse.csr_matrix(feat_n) 
    print("shape of phone {}".format(feat_sPhone.shape)) 
    
    
    #making phone model features
    feat_mod_encodr = pickle.load(open('feat_mod_encodr.sav','rb'))
    #converting model to labels
    feat_modl=brand+model  
    feat_l=[] 
    feat_l.append(feat_modl)  
    feat_rm=feat_mod_encodr.transform(feat_l)  
    feat_n=np.zeros(len(feat_mod_encodr.classes_)) 
    
    feat_n[feat_rm[0]]=1  
    feat_sModel = sparse.csr_matrix(feat_n)  
    print("shape of model {}".format(feat_sModel.shape)) 
    
    #making apps features
    feat_ap_encodr = pickle.load(open('feat_ap_encodr.sav','rb'))
  
    #converting apps to labels
    feat_ind_apps= feat_ap_encodr.transform(feat_ap_id)
    feat_napps = len(feat_ap_encodr.classes_) 
    feat_n=np.zeros(feat_napps) 
    for i in feat_ind_apps: 
        feat_n[i]=1  
    feat_sApps =  sparse.csr_matrix(feat_n) 
    print("shape of apps {}".format(feat_sApps.shape))
     
    #label features
    #converting apps to labels
    feat_lab_encodr = pickle.load(open('feat_lab_encodr.sav','rb')) 
    feat_ind_lab= feat_lab_encodr.transform(feat_labid)
    feat_nlabs = len(feat_lab_encodr.classes_) 
    feat_n=np.zeros(feat_nlabs) 
    for i in feat_ind_lab: 
        feat_n[i]=1  
    feat_sLabs = sparse.csr_matrix(feat_n)
    
    print("shape of labels {}".format(feat_sLabs.shape))
    #model frequency feature
#model frequency feature
    feat_mod_freq = pickle.load(open('feat_mod_freq.pkl','rb'))  
    feat_mfreq=feat_mod_freq[model]
    
    feat_mf_encod =pickle.load(open('feat_mf_encod.sav','rb'))
    feat_l=[] 
    feat_l.append(feat_mfreq) 
    feat_rmf=feat_mf_encod.transform(feat_l) 
    feat_nm_freqs = len(feat_mf_encod.classes_) 
    feat_n=np.zeros(feat_nm_freqs) 
    feat_n[feat_rmf[0]]=1 
    feat_sMod_freq = sparse.csr_matrix(feat_n) 
    print("shape of mfreq {}".format(feat_sMod_freq.shape))
    
    #making brand frequency feature 
    feat_br_freq = pickle.load(open('feat_br_freq.pkl','rb'))
    feat_bfreq=feat_br_freq[brand]
    feat_l=[] 
    feat_l.append(feat_bfreq)
    feat_bf_encod = pickle.load(open('feat_bf_encod.sav','rb'))

    feat_rbf=feat_bf_encod.transform(feat_l) 
    feat_nbfreqs = len(feat_bf_encod.classes_) 
    feat_n=np.zeros(feat_nbfreqs) 
    feat_n[feat_rbf[0]]=1 
    feat_sBr_freq = sparse.csr_matrix(feat_n)
    
    print("shape of bfreq {}".format(feat_sBr_freq.shape)) 
    #number of events feature
    feat_n_eve=len(feat_timestmps) 

    feat_n_eve = (np.log(feat_n_eve)) 
    feat_hrs=[]
    feat_n_eve=feat_n_eve/np.log(243460) 
    
    # hour feature
    for t in feat_timestmps: 
       feat_hrs.append(t.hour) 
    feat_n=np.zeros(24) 
    for hour in feat_hrs:
        feat_n[hour]=1  
    feat_s_hrs = sparse.csr_matrix(feat_n) 
    print("shape of hours {}".format(feat_s_hrs.shape))
    
    
     #preparing tfidf features  
    
    feat_tfv = pickle.load(open('tfv_event.sav','rb'))
    feat_terms=brand+" "+model 
    
    feat_labid=list(feat_labid)
    feat_labid.insert(0,feat_tf) 
    feat_terms=feat_labid  
    feat_terms=list(map(str, feat_tf))
    feat_terms=" ".join(feat_tf)
    
    feat_tfidf_test=feat_tfv.transform([feat_terms]) 
    
    print("shape of tfidf {}".format(feat_tfidf_test.shape))
    Xtest=hstack((feat_sPhone,feat_sModel,feat_sMod_freq,feat_sBr_freq,feat_sApps,feat_sLabs,feat_n_eve,feat_s_hrs,feat_tfidf_test))  
    print(Xtest.shape)
    Xtest=Xtest.tocsc()[:,feat_unq_colmns]
    return Xtest
     
   
    
    
    

    

## First Function for model

In [ ]:
from keras.models import load_model
def feat_f1(X):  
    feat_it = [x for x in range(1,12)] 
    feat_val=iter(X) 
    global feat_unq_col
    feat_res_dct = dict(zip(feat_it, feat_val))  
    feat_eve_id=feat_res_dct[2] 
    feat_timestmps=feat_res_dct[3] 
    feat_timestmps=pd.to_datetime(feat_timestmps)
    feat_ap_id=[feat_res_dct[6]] 
    feat_eve_id=feat_res_dct[2]
    feat_ap_lab=feat_res_dct[9]
    feat_ph=feat_res_dct[10] 
    feat_mod=feat_res_dct[11]
    feat_ypred=np.zeros((1,12))
    if feat_eve_id!=0: 
        Xte=feat_transform(feat_ph,feat_mod) 
        for i in range(1,11): 
            feat_load_mod = pickle.load(open('logistic_'+str(i)+'.sav', 'rb'))
            feat_ypred+=feat_load_mod.predict_proba(Xte)
        feat_ypred=feat_ypred/10 
    else: 
        Xte=feat_trans_eve(feat_ph,feat_mod,feat_ap_id,feat_ap_lab,feat_timestmps,feat_unq_colmns)
        for i in range(1,11): 
            feat_load_mod = load_model('keras_'+str(i)+'.hdf5')
            feat_ypred+=feat_load_mod.predict_proba(Xte)
        feat_ypred=feat_ypred/10 
    return feat_ypred   



In [ ]:
feat_t=feat_gat_test.iloc[:2,:] 
feat_t=feat_test.merge(feat_eve) 
feat_t=feat_test.merge(feat_ap_eve) 
feat_t=feat_test.merge(feat_ap_lab) 
feat_t=feat_test[feat_test.feat_dev_id==feat_test.feat_dev_id[0]]

In [ ]:
feat_test=feat_test.merge(feat_ph) 


In [ ]:
feat_test

,device_id,event_id,timestamp,longitude,latitude,app_id,is_installed,is_active,label_id,phone_brand,device_model
0,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,549,小米,小米note
1,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,710,小米,小米note
2,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,704,小米,小米note
3,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,548,小米,小米note
4,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,172,小米,小米note
...,...,...,...,...,...,...,...,...,...,...,...
1781,1002079943728939269,3127685,2016-05-06 22:05:18,0.0,0.0,3717049149426646122,True,False,548,小米,小米note
1782,1002079943728939269,3127685,2016-05-06 22:05:18,0.0,0.0,3717049149426646122,True,False,183,小米,小米note
1783,1002079943728939269,2145937,2016-05-05 08:28:20,0.0,0.0,-3473060463208403965,True,False,1014,小米,小米note
1784,1002079943728939269,2774265,2016-05-07 09:08:31,0.0,0.0,-3473060463208403965,True,False,1014,小米,小米note


In [ ]:
feat_test=[1002079943728939269, 460577, ['2016-05-03 21:06:29','2016-05-05 08:28:20'], 0.0, 0.0, 5927333115845830913, True, False, [549,710,172], '小米', '小米note']

In [ ]:
import pickle
feat_f1(feat_test)

shape of phone (1, 131)
shape of model (1, 1667)
shape of mfreq (1, 310)
shape of brandfreq (1, 73)
shape of tfidf (1, 1570)


array([[0.05807013, 0.05720962, 0.04567753, 0.05930158, 0.06988216,
        0.05366598, 0.117675  , 0.12713613, 0.08327792, 0.09351835,
        0.12175918, 0.11282642]])

In [ ]:
feat_transform()

## Second function for model

In [ ]:
 
def feat_f1(device_id,phone,model,has_event=0,appid=0,applabel=0,timestamps=0,unique_columns=0,y): 
    feat_ypred=np.zeros((1,12))
    if has_event==0: 
        feat_Xte=feat_transform(feat_dev_id,feat_ph,feat_mod) 
        for i in range(1,11): 
            feat_load_mod = pickle.load(open('logistic_'+str(i)+'.sav', 'rb'))
            feat_ypred+=feat_load_mod.predict_proba(feat_Xte)
        feat_ypred=feat_ypred/10 
    else: 
        feat_Xte=feat_transform(feat_dev_id,feat_ph,feat_mod,feat_appid,feat_ap_lab,feat_timestmps,feat_unq_colmns)
        for i in range(1,11): 
            feat_load_mod = load_model('keras_'+str(i)+'.hdf5')
            feat_ypred+=feat_load_mod.predict_proba(feat_Xte)
        feat_ypred=feat_ypred/10  
        feat_los=log_loss(feat_ypred,y)
    return feat_los   




In [ ]:
from prettytable import PrettyTable
feat_x = PrettyTable()

feat_x.field_names = ["Model", "Parameters", "Score"]

feat_x.add_row(["Logistic Regression", "C=0.02", 2.27])
feat_x.add_row(["Decision Tree","min_sample_split=6 ,max_depth=6", 2.41])
feat_x.add_row(["Xgboost", "'max_depth':6 ,'min_child_weight':5", 2.257])
feat_x.add_row(["Keras 1", "epochs=18 batches=175", 2.24])
feat_x.add_row(["Keras 2 different models for events and no events", "events:{epochs=80 batches=5} noevents:{epochs=30 batches=100}", 2.24])
feat_x.add_row(["Keras 3","epochs=200 batches=5", 2.2378])
feat_x.add_row(["Keras for events and logistic regression for noevents","keras:{epochs=20,batches=200}, lr:{C=0.016} ", 2.23704]) 
feat_x.add_row(["ensemble model","5 model predictions",2.23714])
feat_x.align = "l"
print(feat_x)

+-------------------------------------------------------+---------------------------------------------------------------+---------+
| Model                                                 | Parameters                                                    | Score   |
+-------------------------------------------------------+---------------------------------------------------------------+---------+
| Logistic Regression                                   | C=0.02                                                        | 2.27    |
| Decision Tree                                         | min_sample_split=6 ,max_depth=6                               | 2.41    |
| Xgboost                                               | 'max_depth':6 ,'min_child_weight':5                           | 2.257   |
| Keras 1                                               | epochs=18 batches=175                                         | 2.24    |
| Keras 2 different models for events and no events     | events:{epochs=80 